## Bibliotheken importieren und Modell initialisieren

In [ ]:
import deepwaveform as dwf
import matplotlib.pyplot as plt
import torch
df = next(dwf.load_dataset("../data/mit_uferuebergang.csv"))
df

In [ ]:
ds = dwf.WaveFormDataset(df, classcol=None)                     # Datensatz in passende Form bringen
model = dwf.AutoEncoder(hidden=12)                              # Modell initialisieren
trainer = dwf.Trainer(model, ds, batch_size=4096, epochs=20)    # Trainer mit Datensatz und Modell initialisieren

## Trainieren des Autoencoders

In [ ]:
stats = []
for epoch, result in enumerate(trainer.train_autoencoder(sparsity=0), start=1):
    stats.append(result)
    if epoch%1==0:
        print("epoch=%s E[loss]=%.5f Var[loss]=%.5f" % (str(epoch).zfill(3), 
                                                        result["meanloss"], 
                                                        result["varloss"]))

fig, ax = plt.subplots(1,1)
dwf.plot_training_progress(stats, ax)

## Speichern des Autoencoders

In [ ]:
torch.save(model.state_dict(), "trained_models/autoencoder.pt")

## Datensatz annotieren
Mit der Funktion `annotate_dataframe` des Autoencoders werden neue Spalten hinzugefügt, die die Kodierung und die rekonstruierten Waveforms enthalten.

In [ ]:
model.annotate_dataframe(df,                                    # Der Datensatz, der annotiert werden soll
                         encoding_prefix="hidden_",             # Spaltenpräfix der Kodierung
                         reconstruction_prefix="reconstr_")     # Spaltenpräfix der Rekonstruktion
df

## Visualisierung der echten Waveform vs. Rekonstruktion

In [ ]:
sampled = df.sample(n=1).reset_index()
fig, ax = plt.subplots(1, 1, figsize=(12,6))
dwf.plot_waveforms(sampled, 
                   ax, 
                   class_label_mapping=["Land (True)", "Water (True)"], 
                   class_style_mapping=["g-","b-"],
                   wv_cols=list(map(str, range(64))))
dwf.plot_waveforms(sampled, 
                   ax, 
                   class_label_mapping=["Land (reconstructed)", "Water (reconstructed)"], 
                   class_style_mapping=["g--","b--"],
                   wv_cols=["reconstr_%d" % idx for idx in range(64)])